#Variational Quantum Eigensolver & QAOA / Introduction

##Introduction
The existing quantum computer in 2020 has a lot of error and it looks difficult to run an algorithm with a deep circuit. Instead we usually use a quantum-classical hybrid algorithm called variational algorithm. Here we are going to learn a practical hybrid algorithm on blueqat.

###Overview
Variational Quantum Eigensolver(VQE) is developed as an alternate algorithm of "Quantum Phase Estimation" which solve an eigenvalue of Matrix from a state vector.

The existing quantum computer has a lot of error on the result, so VQE run many short circuit to avoid errors and optimize the parameter of circuit. By using this method VQE can solve the problem on Eigenvector and Eigenvalue on quantum-classical hybrid.

###Reference
A variational eigenvalue solver on a quantum processor  
Alberto Peruzzo, Jarrod McClean, Peter Shadbolt, Man-Hong Yung, Xiao-Qi Zhou, Peter J. Love, Alán Aspuru-Guzik, Jeremy L. O'Brien  
(Submitted on 10 Apr 2013)
https://arxiv.org/abs/1304.3061

##Eigenvalue and eigenvector
The problem VQE solves is eigenvalue and eigenvector. If a matrix is given and one can find its eigenvector, many problems can be solved by using this method. 

Now with the given matrix $H$, Eigenvalue$E_0$ and Eigenvector$\mid \psi \rangle$ satisfy this equation.

$$H \mid \psi \rangle = E_0 \mid \psi \rangle$$

We want to find this $E_0$ and $\psi$ using this algorithm. $H$ is called Hamiltonian and $\mid \psi \rangle$ is called State Vector (Wave function).

##What solved?
VQE is a very general algorithm to solve the eigenvalue and eigenvector problem. But actually it is very difficult to find the eigenstate from a variety of parameters. Practially without any relation between hamiltonian $H$ and state vector $\mid \psi \rangle$ provided by ansatz, we cannot solve the problem efficiently.

Now we have two types of problems which this relation is found, "quantum chemistry" and "combinatorial optimization problem". These problems show relation between ansatz and hamiltonian.

To get the eigenstate we make an quantum circuit and get the eigenstate $\mid \psi \rangle$. We calle it ansatz. Quantum chemistry has UCC ansatz and combinatorial optimization has QAOA(Quantum Approximate Optimization Algorithm) ansatz.

These two are similar but especially when you use this ansatz on optimization problem it is called QAOA.

Now we try to use these VQE and QAOA algorithm on blueqat SDK ( https://github.com/blueqat/blueqat )

##Variational Algorithm2-1（Theory）
Now we start a basic theory. VQE and QAOA algorithms are so called variational algorithm and can run almost the same steps. The point to learn is

1. About Hamiltonian
2. About Ansatz

Let's see a simple example.

##Installing blueqat
We use blueqat

https://github.com/Blueqat/Blueqat

We can install it using this command,

In [0]:
!pip3 install blueqat

     |████████████████████████████████| 51kB 1.6MB/s 


##2-1-1 qubit and state vector
Qubit is used in quantum computer and can take a superposition of |0> state and |1> state. It can be written in a form of vertical vector.

$$
\mid 0 \rangle =
\begin{bmatrix}
1 \\
0
\end{bmatrix},
\mid 1 \rangle =
\begin{bmatrix}
0 \\
1
\end{bmatrix}
$$

By using this we can write a state vector as a superposition of two states.


$$
\mid \psi \rangle = \alpha \mid 0 \rangle + \beta \mid 1 \rangle =
\alpha 
\begin{bmatrix}
1 \\
0
\end{bmatrix}
+ \beta
\begin{bmatrix}
0 \\
1
\end{bmatrix}
=
\begin{bmatrix}
\alpha \\
\beta
\end{bmatrix}
$$

$\mid \psi \rangle$ is called state vecor and shows the state of qubit. $\alpha$ and $\beta$ are complex number and called probability amplituted. We can get the value of $|\alpha|^2$ and $|\beta|^2$ by measurement. $|\alpha|^2$ and $|\beta|^2$ satisfy,

$$
|\alpha|^2+|\beta|^2 = 1
$$

If we calcurate 100times and get 0 as 45times and 1 as 55times, expectation value of $|\alpha|^2$ is 0.45 and that of $|\beta|^2$ is 0.55. Basically we cannot get the direct state vector, we have to get it from samples.

##2-1-2 Operation of state vecotor and quantum gate
To operate a state vector, we use quantum gate. In a variational algorithm we use pauli gates, rotation gates and hadamard gate.

Pauli gate consists of 3 operators of XYZ. And these are 180 degree rotation around each axis. Each pauli operators are,

$$
X=
\begin{bmatrix}
0&1 \\
1&0
\end{bmatrix},
Y=
\begin{bmatrix}
0&-i \\
i&0
\end{bmatrix},
Z=
\begin{bmatrix}
1&0 \\
0&-1
\end{bmatrix}
$$

RX,RY,RZ gate are rotation gate operators,

$$
Rx(\theta) = \left( \begin{array}{cc} \cos\left(\frac{\theta}{2}\right) &
-i\sin\left(\frac{\theta}{2}\right)\\
-i\sin\left(\frac{\theta}{2}\right) &
\cos\left(\frac{\theta}{2}\right) \end{array} \right),
Ry(\theta) = \left( \begin{array}{cc} \cos\left(\frac{\theta}{2}\right) &
-\sin\left(\frac{\theta}{2}\right)\\
\sin\left(\frac{\theta}{2}\right) &
\cos\left(\frac{\theta}{2}\right) \end{array} \right),
Rz(\theta) = \left( \begin{array}{cc} e^{-i\frac{\theta}{2}} & 0\\ 0 & e^{i\frac{\theta}{2}} \end{array} \right)
$$

Hadamard gate is,

$$
H =
\frac{1}{\sqrt{2}} 
\begin{bmatrix}
1&1 \\
1&-1
\end{bmatrix}
$$

Basically Hadamard gate convert data between X axis and Z axis. By applying these operation on state vector, the state vector can be changed.

$$
X
\begin{bmatrix}
1 \\
0
\end{bmatrix}
=
\begin{bmatrix}
0&1 \\
1&0
\end{bmatrix}
\begin{bmatrix}
1 \\
0
\end{bmatrix}
=
\begin{bmatrix}
0 \\
1
\end{bmatrix}
$$

##2-1-3 State vector of multiple qubits and tensor product
We saw about 1qubit, now we see multiple qubits. To get the state vector on multiple qubits we use tensor product.

$$
\begin{bmatrix}
a \\
b
\end{bmatrix}
\otimes
\begin{bmatrix}
c \\
d
\end{bmatrix}
=
\begin{bmatrix}
a*
\begin{bmatrix}
c\\
d
\end{bmatrix}
 \\
b*
\begin{bmatrix}
c\\
d
\end{bmatrix}
\end{bmatrix}
=
\begin{bmatrix}
ac \\
ad \\
bc \\
bd
\end{bmatrix}
$$

Over 3qubits we use the same method.

$$
\begin{bmatrix}
a \\
b
\end{bmatrix}
\otimes
\begin{bmatrix}
c \\
d
\end{bmatrix}
\otimes
\begin{bmatrix}
e \\
f
\end{bmatrix}
=
\begin{bmatrix}
ac *
\begin{bmatrix}
e \\
f
\end{bmatrix} \\
ad*
\begin{bmatrix}
e \\
f
\end{bmatrix} \\
bc*
\begin{bmatrix}
e \\
f
\end{bmatrix} \\
bd*
\begin{bmatrix}
e \\
f
\end{bmatrix}
\end{bmatrix}
=
\begin{bmatrix}
ace\\
acf\\
ade\\
adf\\
bce\\
bcf\\
bde\\
bdf
\end{bmatrix}
$$

The gate operators can use tensor product to get the unitary matrix on multiple qubits.

$$
\begin{bmatrix}
a&b \\
c&d
\end{bmatrix}
\otimes
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix} \\
=
\begin{bmatrix}
a*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
&b*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
\\
c*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
&d*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
\end{bmatrix}
=
\begin{bmatrix}
ae&af&be&bf\\
ag&ah&bg&bh\\
ce&cf&de&df\\
cg&ch&dg&dh
\end{bmatrix}
$$


##2-1-4 Quantum variational method
It is very difficult to directly get the eigenvalue now, we calculate the expectation value of it using variational method.

$$
\langle \Psi (\theta) \mid H \mid \Psi (\theta) \rangle \geq E_0
$$

It shows that the expectation value of given hamiltonian $H$ is always over the minimum eigenvalue. The hamiltonian is usually a given formulation, we can just operate on state vector to find a better one closer to the eigenvector.

##2-1-5 Steps of variational algorithm
The step to use the variational algorithm is,

1. Prepare an ansatz with parameter$\theta$ (quantum)
2. Calculate expectation value $\langle \Psi (\theta) \mid H \mid \Psi (\theta) \rangle$ of hamiltonian $H$ by the measurement.(classical)
3. By the classical optimization, find the minimum value of expectation value and give another parameter$\theta$ to the parameterized circuit.(classical)

The point is that usually given hamiltonian $H$ is hermite matrix and can be decomposited into linear combination of unitary matrix.


##2-1-6 ansatz
Ansatz is a parameterized quantum circuit with rotation gate which create the state vector. The state vector provides measurement value to estimate the expectation value of Hamiltonian.

##2-1-7 Hamiltonian
Hamiltonian is a matrix consists of XYZ pauli gate and I identity matrix. Now the variational algorithm is to get the expectation value of eigenvalue and state vector as the eigenstate of this matrix.

##2-1-8 Expectation value of hamiltonian
If a state vector$\mid \psi \rangle$ and hamiltonian $H$ is given, the expectation value of hamiltonian is simply,

$$
\langle \psi (\theta) \mid H \mid \psi (\theta) \rangle
$$

##2-1-9 Example: Calculate the expectation value of Z using ansatz
By using an ansatz, the expectation value of Z is calculated as,

$$
\langle \psi \mid H \mid \psi \rangle = 
\begin{bmatrix}
\alpha^* & \beta^*
\end{bmatrix}
\begin{bmatrix}
1&0\\
0&-1
\end{bmatrix}
\begin{bmatrix}
\alpha\\
\beta
\end{bmatrix}
= |\alpha|^2 - |\beta|^2
$$

$|\alpha|^2$ is the probability to get 0 and $|\beta|^2$ is the probability to get 1.

##2-1-10 Example2: Calculate the expectation value of X using ansatz
The expecation value of $H = X$ is,

$$
\langle \psi \mid H \mid \psi \rangle = 
\begin{bmatrix}
\alpha^* & \beta^*
\end{bmatrix}
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\begin{bmatrix}
\alpha\\
\beta
\end{bmatrix}
= \alpha^* \beta + \alpha \beta^*
$$

But we cannot get these amplitude directly from the measurement.



##2-1-11 deformation of hamiltonian
If the hamiltonian$H$ is $Z$, we can easily get the expectation value. But if $H$ is X or Y it is difficult to get the expectation value. So we need a deformation of the hamiltonian. X gate satisfy,

$$
X = HZH
$$

And we use it for hamiltonian

$$
\langle \psi \mid X \mid \psi \rangle \\
= \langle \psi \mid HZH \mid \psi \rangle\\
= \langle \psi' \mid Z \mid \psi' \rangle
$$

H can be included into the state vector, and Y gate also satisfy,

$$
Y = RX(-\pi/2) Z RX(\pi/2)
$$

And we get,

$$
\langle \psi \mid Y \mid \psi \rangle \\
= \langle \psi \mid RX(-\pi/2) Z RX(\pi/2) \mid \psi \rangle\\
= \langle \psi'' \mid Z \mid \psi'' \rangle
$$

Actually we can realize it by just putting H or RX gate before the measurement. 


##2-1-12　Linear combination
The hamiltonian is usually given in Hermite matrix and it is a linear combination of unitary matrix. This can be decomposited into,

$$
\langle \psi \mid aH_1 + bH_2 \mid \psi \rangle \\ = \langle \psi \mid aH_1 \mid \psi \rangle + \langle \psi \mid bH_2 \mid \psi \rangle \\ = a\langle \psi \mid H_1 \mid \psi \rangle + b\langle \psi \mid H_2 \mid \psi \rangle
$$

Using this a complicated matrix like,

$$
H = 1.2 X_0 Z_2 + 2.5 Z_0 X_1 Y_2 - 3.4 Z_2 X_1
$$

can be calculated like,

$$
\langle \psi \mid 1.2 X_0 Z_2 + 2.5 Z_0 X_1 Y_2 - 3.4 Z_2 X_1 \mid \psi \rangle\\
= 1.2\langle \psi \mid X_0 Z_2 \mid \psi \rangle + 2.5 \langle \psi \mid Z_0 X_1 Y_2\mid \psi \rangle - 3.4 \langle \psi \mid Z_2 X_1 \mid \psi \rangle
$$


##2-1-13 Example: random hamiltonian and ansatz
If we don't see about the relation between Hamiltonian $H$ and Ansatz, it would be a very difficult task to solve. Here we use only 1qubit to find the minimum eigenvalue of a very simple hamiltonian $H$.

```python
from blueqat.pauli import X, Y, Z, I

h = 1.23 * I - 4.56 * X(0) + 2.45 * Y(0) + 2.34 * Z(0)
```

This is a very simple hamiltonian with just 1qbit. The number after the Pauli operator denotes the number of qubit. This hamiltonian is written in a matrix expression as,

```python
[[ 3.57,   -4.56-2.45j]
 [-4.56+2.45j, -1.11  ]]
```

And to find the minimum expectation value of this hamiltonian, we use an ansatz. This time we just use 1qbit and need just 2 angle parameters.

```python
Circuit().ry(a)[0].rz(b)[0]
```

We apply two rotation operators on 0th qubit. Ry gate with angle a and Rz gate with angle b on 0th qubit. If we can find a good value of a and b, the hamiltonian takes a good expectation value of Hamiltonian.

Now we have the code as,



In [0]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.vqe import AnsatzBase, Vqe

class OneQubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian.to_expr(), 2)
        self.step = 1

    def get_circuit(self, params):
        a, b = params
        return Circuit().ry(a)[0].rz(b)[0] #making ansatz with 2 parameters


# Hamiltonian
h = 1.23 * I - 4.56 * X(0) + 2.45 * Y(0) + 2.34 * Z(0)
runner = Vqe(OneQubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# For check
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-4.450814236864202
Result by numpy
-4.450818602983201


And we get the very similar value as the actual eigenvalue. Normally we have specific rules on Hamiltonian and Ansatz. Blueqat do most of these formulation and calculation automatically.

In [0]:
from blueqat import vqe
from blueqat.pauli import Z

H = Z(0)*Z(1) #Hamiltonian
step = 2

result = vqe.Vqe(vqe.QaoaAnsatz(H, step)).run()
print(result.most_common(12))

(((1, 0), 0.499883822557386), ((0, 1), 0.499883822557386), ((0, 0), 0.00011617744261394092), ((1, 1), 0.00011617744261394092))


Now we see an actual practical prolems.

1. Quantum Chemistry on VQE
2. Combinatorial Optimization on QAOA